In [1]:
import pandas as pd
pd.set_option('display.max_columns', 35)
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_rows", 500)

### Parsing `txt` to `parquet` for selected 676 cik and does some cleaning with `pandera`
* Casts `cusip` to upper case and cleans empty/zero value or shares
* filter outs `cusip` that are non-8 char 

In [ ]:
%%time
# for selected 676 cik, parse txt filings to parquet
import pathlib
from pathlib import Path
from utils_data_cleaning import end_to_end_parsing

selected_cik = [2230,3520,5272,7195,7789,9015,10742,14661,16972,18349,18748,19475,19617,21175,22657,24386,35442,35527,36066,36104,36644,36966,38777,39263,40417,40545,44365,45319,49205,50863,51762,51812,51964,52234,53417,59558,59951,60086,61227,67698,70858,71210,71259,72971,73124,80255,84616,89014,92230,93751,98758,102212,102909,105495,108572,200217,201772,216851,276101,310051,312348,313028,313807,314949,314957,314984,315014,315032,315038,315054,315066,315080,315157,315297,315498,316011,318989,320335,320376,351051,351173,351262,354204,356264,700529,704051,707179,712537,713676,714142,720672,723204,728083,728100,728618,732905,733020,740272,740913,741073,743127,750641,754811,757657,759944,762152,763212,763848,764068,764106,764112,764529,764532,765443,769317,769954,769963,775368,776867,778963,779519,788714,790354,790502,791191,791490,796848,799003,799004,801051,806097,807249,807985,808722,809339,809443,810265,810384,810386,810672,810716,811360,811454,813917,813933,814133,814375,816788,819535,820027,820123,820124,820289,820478,820743,821197,822581,823621,825293,829407,831001,831571,836372,837592,842782,842941,846222,846633,846788,846797,850401,850529,852743,854157,857508,859872,860486,860561,860580,860585,860643,860644,860645,860662,860748,860828,860857,861176,861177,861462,861787,862469,866361,866842,868491,869178,869179,869353,869367,872080,872163,872259,872573,872732,873630,874791,877134,877338,878228,881432,883511,883677,883782,883790,883803,883961,883965,884300,884314,884414,884423,884541,884546,884548,884566,884589,885062,885415,886982,887402,887777,887818,889232,891287,891478,893738,894205,894300,894309,895213,895421,897070,897378,897599,898358,898382,898399,898413,899211,900169,900529,900973,902219,902367,902464,902584,903064,903944,903947,903949,905567,905591,905608,906304,908195,909151,909661,911274,912938,914933,914976,915287,915325,916542,917579,918893,919079,919185,919192,919458,919489,919497,919530,919538,919859,920440,920441,921531,921669,922127,922439,922898,922940,923093,923116,923469,924166,924171,924181,926688,926833,926834,928047,928196,928566,928568,928633,930441,931097,932024,932974,933429,934639,934999,936698,936753,936936,936941,936944,937394,937522,937589,937615,937760,937886,938076,938206,938487,938582,938592,938759,939219,940445,941560,943719,944234,944804,945625,945631,947822,947996,948518,948669,949012,949509,949615,949623,949853,1000097,1000742,1002152,1002672,1002784,1004244,1005354,1005607,1005817,1006364,1006378,1006407,1006435,1007280,1007399,1007524,1008322,1008877,1008894,1008895,1008929,1008937,1009003,1009005,1009012,1009016,1009022,1009076,1009207,1009209,1009232,1009254,1009258,1009262,1010873,1010911,1011443,1011659,1013234,1013536,1013538,1013701,1014306,1014315,1014736,1014738,1015079,1015083,1015086,1015308,1016150,1016287,1016683,1016972,1017115,1017645,1017918,1018331,1018674,1018825,1019231,1020066,1020317,1020580,1020585,1020617,1020918,1021008,1021117,1021223,1021249,1021258,1021642,1021926,1023279,1024716,1025421,1026200,1026710,1027451,1027796,1027817,1029160,1030618,1030815,1031972,1032814,1033225,1033427,1033475,1033505,1033974,1033984,1034184,1034196,1034524,1034541,1034546,1034549,1034642,1034771,1034886,1035350,1035463,1035912,1036248,1036325,1037389,1037558,1037763,1037792,1038661,1039565,1039807,1040190,1040197,1040198,1040210,1040273,1040592,1040762,1041241,1041885,1042046,1044207,1044797,1044905,1044924,1044929,1044936,1046187,1047339,1048921,1049648,1049650,1050442,1050463,1050470,1051359,1052100,1053013,1053054,1053055,1054074,1054425,1054522,1054554,1054677,1055290,1055544,1055963,1055964,1055966,1056053,1056288,1056466,1056488,1056491,1056515,1056516,1056527,1056549,1056559,1056581,1056593,1056807,1056821,1056825,1056827,1056831,1056859,1056958,1056973,1057395,1057439,1058022,1058470,1058800,1059187,1061186,1061768,1062938,1065349,1065350,1066816,1067324,1067926,1067983,1068829,1070134,1071483,1072843,1074027,1074034,1074266,1074273,1076598,1077148,1077583,1078013,1078246,1078658,1078841,1079112,1079114,1079736,1079738,1079930,1080071,1080107,1080117,1080132,1080166,1080171,1080173,1080197,1080201,1080351,1080374,1080380,1080381,1080382,1080386,1080493,1080523,1080628,1080818,1081019,1081198,1082020,1082215,1082327,1082339,1082461,1082491,1082509,1082621,1082917,1083323,1083340,1084207,1084208,1084683,1085041,1085163,1085227,1085601,1085936,1086477,1086483,1086611,1086619,1086762,1086763,1088859,1088875,1088950,1089707,1089755,1089911,1089991,1090413,1091561,1091860,1091923,1092203,1092290,1092351,1092903,1093276,1093589,1094584,1094749,1095836,1096783,1097218,1097278,1097833,1100710,1101250,1102062,1102578,1102598,1103245,1103738,1103804,1103882,1103887,1104186,1104329,1104366,1105468,1105471,1105497,1105837,1105863,1105909,1106129,1106191,1106500,1106505,1106832,1107261,1107310,1108893,1108965,1108969,1109147,1110806,1113629,1114618,1114739,1114928,1115941,1116247,1125727,1125816,1129770,1133219,1134152,1140334,1140771,1142031,1142062,1158583,1389426,1398739,1469219]
TR_00_TXT_SEC_APP = Path('/Volumes/fanpc/app_data/sec_apps_data/speed_test/filings_13f_full/filings')
TR_02_TEST_676_CIK_PARQ_SEC_APP = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_TEST_676_CIK_PARQ_SEC_APP')
TR_02_FAILURE_CASES_PARQ_SEC_APP = Path(r"/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_FAILURE_CASES_PARQ_SEC_APP")

filings_parq = TR_02_TEST_676_CIK_PARQ_SEC_APP.glob("*.parquet")
f = TR_00_TXT_SEC_APP
l = selected_cik
subdirs = list(f.glob("*"))
subdirs = [d for d in subdirs if d.is_dir() and int(d.name) in l]

for subdir in subdirs: # [-20:-1]
    filings_txt = list(subdir.glob("*.txt"))
    for file in filings_txt:
        file_exists = list(TR_02_TEST_676_CIK_PARQ_SEC_APP.glob(f"{subdir.stem}-{file.stem}*"))
        # Check if the file exists
        if len(file_exists) > 0:
            continue
        else:
            # print("File does not exist. Converting to parquet...")
            end_to_end_parsing(file, TR_02_TEST_676_CIK_PARQ_SEC_APP, TR_02_FAILURE_CASES_PARQ_SEC_APP)


### Parsing `txt` to `parquet` for **ALL** `cik` and does some cleaning with `pandera`
* Casts `cusip` to upper case and cleans empty/zero value or shares
* filter outs `cusip` that are non-8 char 

In [ ]:
%%time
# Parse txt filings to parquet for ALL cik
import pathlib
from pathlib import Path
from utils_data_cleaning import end_to_end_parsing

TR_00_TXT_SEC_APP = Path('/Volumes/fanpc/app_data/sec_apps_data/speed_test/filings_13f_full/filings')
TR_02_ALL_PARQ_SEC_APP = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_ALL_PARQ_SEC_APP')
TR_02_FAILURE_CASES_PARQ_SEC_APP = Path(r"/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_FAILURE_CASES_PARQ_SEC_APP")

# all_filings_parq = TR_02_ALL_PARQ_SEC_APP.glob("*.parquet")
subdirs = list(TR_00_TXT_SEC_APP.glob("*"))

for subdir in subdirs: # [-20:-1]
    filings_txt = list(subdir.glob("*.txt"))
    for file in filings_txt:
        file_exists = list(TR_02_ALL_PARQ_SEC_APP.glob(f"{subdir.stem}-{file.stem}*"))
        # Check if the file exists
        if len(file_exists) > 0: continue
        else: end_to_end_parsing(file, TR_02_ALL_PARQ_SEC_APP, TR_02_FAILURE_CASES_PARQ_SEC_APP)


In [ ]:
file

### Parse individual `txt` file to `parquet`

In [ ]:
import pathlib
from pathlib import Path
from utils_data_cleaning import end_to_end_parsing

file = Path(r'/Volumes/fanpc/app_data/sec_apps_data/speed_test/filings_13f_full/filings/902528/0001178913-20-002231.txt')
test_parq = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/test_parq')
test_failure = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/test_failure')

df_test = end_to_end_parsing(file, test_parq, test_failure)
# .sort_values(by='value', ascending=True)

In [ ]:
# df_test.head()
# df_test.info()

In [ ]:
%%time 
from datetime import datetime
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pandera as pa

############################################################################
############################################################################

## Function to end-to-end parse the raw `txt` filings, write to a pandas dataframe
## and save to a `parquet` file. Only `xml` based filings are dealt with. 
## The filings prior `xml` format are ignored and instead we use the `csv` filings from dropbox

## no division by 1000
## cusip transformed to upper case
## uses `pandera` to remove `cusip` != length of 9 (as non compliant)
## uses `pandera` to remove `values` or `shares` != 0 (as non compliant)
## the code writes bad data into a separate dataframes per filings
## transformation where we group multiple `cusip` into one only after we have removed 
# the ones with zeros in `value` or `shares` 


def end_to_end_parsing(file_path, directory_parquet, failures_parq_dir, data_load_run, cutoff_date = '2023-02-17'):
    """
    func that takes a txt filing's path and parses it to a parquet file
    in the `directory_parquet`. It uses `pandera` lazy mode to filter out
    bad cusip or empty/nan/zero shares or values. Bad records are written
    to the `failures_parq_dir` for a later analysis or correction.
    
    """
    def work_content_extract(file_path):
        """extracting 1. xml or not flag, 2. xml part of filings, SEC header part"""
        open_file = open(file_path, "r")
        file_content = open_file.read()
        open_file.close()

        # find block of text between two words/tags/strings
        start_header = "<SEC-HEADER>"
        end_header = "</SEC-HEADER>"
        block_header_search = re.compile(f"{start_header}.*?{end_header}", re.DOTALL)
        pages_header = re.findall(block_header_search, file_content)

        # check if file is XML or not
        xml_tag_search = re.compile(r"<xml>", flags=re.IGNORECASE)
        if xml_tag_search.search(file_content):
            xml_flag = "YES"

            # xml block search
            start_xml = "<XML\>"
            end_xml = "<\/XML\>"
            block_xml_search = re.compile(f"{start_xml}(.*?){end_xml}", re.DOTALL)
            pages_xml = re.findall(block_xml_search, file_content)

            if len(pages_xml) == 0:
                reportType = ""
                document0 = ""
                document1 = ""
            elif len(pages_xml) == 1:
                document0 = bs(pages_xml[0], "xml")
                coverPage = document0.find("coverPage")
                reportType = coverPage.find("reportType").text.strip()
                document1 = ""
            if len(pages_xml) > 1:
                document0 = bs(pages_xml[0], "xml")
                coverPage = document0.find("coverPage")
                reportType = coverPage.find("reportType").text.strip()
                document1 = bs(pages_xml[1], "xml")

        else:
            xml_flag = "NO"
            pages_xml = []
            reportType = ""
            document1 = ""

        return pages_header, pages_xml, xml_flag, reportType, document1

    def parse_institutionalInvestorInfo(file_path):
        dataDictionary = dict()
        dataDictionary["edgar_path"] = file_path
        periodOfReport = datetime.strptime(
            re.compile(r"(?<=CONFORMED PERIOD OF REPORT:).*")
            .search(pages_header[0])
            .group(0)
            .strip(),
            "%Y%m%d",
        ).date()
        report_Year, report_Quarter = (
            periodOfReport.year,
            (periodOfReport.month - 1) // 3 + 1,
        )

        dataDictionary["accessionNumber"] = (
            re.compile(r"(?<=ACCESSION NUMBER:).*")
            .search(pages_header[0])
            .group(0)
            .strip()
        )
        dataDictionary["cikManager"] = (
            re.compile(r"(?<=CENTRAL INDEX KEY:).*")
            .search(pages_header[0])
            .group(0)
            .strip()
        )
        dataDictionary["managerName"] = (
            re.compile(r"(?<=COMPANY CONFORMED NAME:).*")
            .search(pages_header[0])
            .group(0)
            .strip()
        )
        dataDictionary["periodOfReport"] = periodOfReport
        dataDictionary["report_Quarter"] = report_Quarter
        dataDictionary["report_Year"] = report_Year
        dataDictionary["submissionType"] = (
            re.compile(r"(?<=CONFORMED SUBMISSION TYPE:).*")
            .search(pages_header[0])
            .group(0)
            .strip()
        )
        dataDictionary["filedAsOfDate"] = datetime.strptime(
            re.compile(r"(?<=FILED AS OF DATE:).*")
            .search(pages_header[0])
            .group(0)
            .strip(),
            "%Y%m%d",
        ).date()
        dataDictionary["xml_flag"] = xml_flag
        dataDictionary["created_at"] = datetime.now()
        dataDictionary["updated_at"] = datetime.now()

        # dataDictionary = dict()
        dataDictionary["edgar_path"] = file_path
        if xml_flag == "NO":

            dataDictionary["isAmendment"] = ""
            dataDictionary["amendmentType"] = ""
            dataDictionary["entryTotal"] = int("0" + "")
            dataDictionary["valueTotal"] = float("0.0" + "")
            dt = pd.DataFrame.from_dict([dataDictionary])

        else:
            if len(pages_xml) == 0:
                dataDictionary["isAmendment"] = ""
                dataDictionary["amendmentType"] = ""
                dataDictionary["entryTotal"] = int("0" + "")
                dataDictionary["valueTotal"] = float("0.0" + "")
                dt = pd.DataFrame.from_dict([dataDictionary])

            else:
                document = bs(pages_xml[0], "xml")
                # get sections
                coverPage = document.find("coverPage")
                summaryPage = document.find("summaryPage")
                # get data
                if coverPage.find("isAmendment") is not None:

                    if coverPage.find("isAmendment").text.strip() == "true":
                        dataDictionary["isAmendment"] = coverPage.find(
                            "isAmendment"
                        ).text.strip()
                        if coverPage.find("amendmentType"):
                            dataDictionary["amendmentType"] = coverPage.find(
                                "amendmentType"
                            ).text.strip()
                        else:
                            dataDictionary["amendmentType"] = ""
                    else:
                        dataDictionary["isAmendment"] = coverPage.find(
                            "isAmendment"
                        ).text.strip()
                        dataDictionary["amendmentType"] = ""
                else:
                    dataDictionary["isAmendment"] = ""
                    dataDictionary["amendmentType"] = ""

                if summaryPage is not None:
                    if summaryPage.find("tableEntryTotal").text.strip():
                        dataDictionary["entryTotal"] = int(
                            float(summaryPage.find("tableEntryTotal").text.strip() + "0.0")
                        )
                    else:
                        dataDictionary["entryTotal"] = int("0" + "")


                    if summaryPage.find("tableValueTotal").text.strip():
                        dataDictionary["valueTotal"] = float(
                        summaryPage.find("tableValueTotal").text.strip()
                    + "0.0")

                    else: dataDictionary["valueTotal"] = float("0.0" + "")
                else:
                    dataDictionary["entryTotal"], dataDictionary["valueTotal"] = int(
                        "0" + ""
                    ), float("0.0" + "")

                # create dataframe
                dt = pd.DataFrame.from_dict([dataDictionary])

        return dt

    def parse_institutionalInvestorPortfolio(file_path):
        check = re.compile("13F HOLDINGS REPORT|13F COMBINATION REPORT")
        if check.search(reportType) is not None and len(pages_xml) > 1:

            portfolio = list()
            # find all securities held

            securities = document1.find_all("infoTable")
            for row in securities:
                portfolioDict = dict()
                portfolioDict["edgar_path"] = file_path
                portfolioDict["cusip"] = row.find("cusip").text.strip()
                portfolioDict["nameOfIssuer"] = row.find("nameOfIssuer").text.strip()
                portfolioDict["titleOfClass"] = row.find("titleOfClass").text.strip()
                portfolioDict["sharesValue"] = float(
                    row.find("value").text.strip()
                )
                portfolioDict["sharesHeldAtEndOfQtr"] = int(
                    float(row.find("sshPrnamt").text.strip())
                )

                try:
                    portfolioDict["sharePriceAtEndOfQtr"] = round(
                        portfolioDict["sharesValue"]
                        / portfolioDict["sharesHeldAtEndOfQtr"],
                        2,
                    )
                except ZeroDivisionError:
                    portfolioDict["sharePriceAtEndOfQtr"] = float(int("0" + ""))

                portfolioDict["shares_bonds"] = row.find("sshPrnamtType").text.strip()

                if row.find("putCall") is not None:
                    portfolioDict["putCall"] = row.find("putCall").text.strip()
                else:
                    portfolioDict["putCall"] = ""

                portfolio.append(pd.DataFrame.from_dict([portfolioDict]))

            # concatanate secutires
            dtPortfolio = pd.concat(portfolio, sort=False, ignore_index=True)
        else:
            dtPortfolio = pd.DataFrame.from_dict(
                [
                    {
                        "cusip": "",
                        "nameOfIssuer": "",
                        "titleOfClass": "",
                        "sharesValue": float("0.0" + ""),
                        "sharesHeldAtEndOfQtr": int("0" + ""),
                        "sharePriceAtEndOfQtr": float("0.0" + ""),
                        "shares_bonds": "",
                        "putCall": "",
                        "edgar_path": file_path,
                    }
                ]
            )

        return dtPortfolio

    pages_header, pages_xml, xml_flag, reportType, document1 = work_content_extract(
        file_path
    )
    df1, df2 = parse_institutionalInvestorInfo(
        file_path
    ), parse_institutionalInvestorPortfolio(file_path)

    data = pd.merge(df1, df2, on="edgar_path")
    data = data.assign(dsource='sec_app')    
    data = data.rename(columns={'accessionNumber': 'accession_number', "cikManager": 'cik', "managerName":'cik_name','xml_flag':'type',
           "periodOfReport": 'rdate', "submissionType": 'submission_type', "filedAsOfDate":'fdate', "entryTotal": "entry_total",
           "valueTotal": "value_total", 'putCall':'put_call', "sharesValue":'value', "sharesHeldAtEndOfQtr":'shares',
           "securityType": "security_type", 'titleOfClass':'title_of_class', 'nameOfIssuer':'name_of_issuer' , "edgar_path":'file'})

    column_names = [
        "accession_number",
        "cik",
        "cik_name",
        "rdate",
        "submission_type",
        "fdate",
        "entry_total",
        "value_total",
        "cusip",
        "name_of_issuer",
        "title_of_class",
        "value",
        "shares",
        "shares_bonds",
        "put_call",
        "type",
        "file",
        'dsource',
        "created_at",
        'updated_at',
        'data_load_run'
    ]
    data = data.astype(
        {
            "accession_number": str,
            "cik": "Int64",
            'cik_name': str,
            "rdate": "datetime64[ns]",
            "submission_type": str,
            "fdate": "datetime64[ns]",
            "entry_total": "Int64",
            "value_total": "float64",
            "cusip": str,
            "name_of_issuer": str,
            "title_of_class": str,
            "value": "float64",
            "shares": "float64",
            "shares_bonds": str,
            "put_call": str,
            "type": str,
            "file": str,
            'dsource': str,
            "created_at": "datetime64[ns]",
            "updated_at": "datetime64[ns]"
        }
    )
    attributes = {
        "cik": "first",
        "cik_name": "first",
        "rdate": "first",       
        "fdate": "first",
        "cusip": "first",
        "name_of_issuer": "first",        
        "value": "sum",
        "shares": "sum",
        "title_of_class": "first",
        "shares_bonds": "first",
        "put_call": "first",
        "accession_number": "first",
        "submission_type": "first",
        "type": "first",
        "file": "first",
        'dsource': 'first',
        "entry_total": "first",
        "value_total": "first",
        "created_at": "first",
        "updated_at": "first",
        'data_load_run': 'first'
    }
    data = data.assign(cusip=data.cusip.str.upper(),
                       data_load_run=data_load_run)
    data = data.reindex(columns=column_names)
   
    ## pandera code
    validation_schema = pa.DataFrameSchema({
    "cusip": pa.Column(str,
                       pa.Check(lambda s: s.str.len() == 9),
                       required=True, nullable=False),
    "value":  pa.Column(float, pa.Check(lambda s: s != 0.0), required=True, nullable=False),
    "shares": pa.Column(float, pa.Check(lambda s: s != 0.0), required=True, nullable=False)
        })
    if data.head(1).type.squeeze() != 'NO' and data.head(1).fdate.squeeze() > pd.to_datetime(cutoff_date,format='%Y-%m-%d'):
        try:
            validation_schema.validate(data, lazy=True)
            if not data.empty:
                # group cusips
                df2 = data.groupby(["cusip"], as_index=False).agg(attributes)
                df2.to_parquet(
                    os.path.join(
                        directory_parquet,
                        f"{df2.head(1).cik[0]}-{df2.head(1).accession_number[0]}-{df2.head(1).fdate[0].strftime('%Y-%m-%d')}.parquet",
                                    )
                                )

                # return df2
            else: pd.DataFrame()

        except pa.errors.SchemaErrors as e:
            failure_cases = e.failure_cases
            failure_cases = (failure_cases.assign(df_file=data.file,
                                                 df_cik=data.cik,
                                                 df_accession_number=data.accession_number,
                                                 df_rdate=data.rdate,
                                                 df_fdate=data.fdate,
                                                 df_value=data.value)).astype({'failure_case':str})
            # print(failure_cases)
            failure_cases.to_parquet(Path.joinpath(failures_parq_dir, \
                                                   f"bad-{data.head(1).cik[0]}-{data.head(1).accession_number[0]}-{data.head(1).fdate[0].strftime('%Y-%m-%d')}.parquet"))
            data = data[~data.index.isin(failure_cases["index"])]
            
            if not data.empty:
                # group cusips
                df2 = data.groupby(["cusip"], as_index=False).agg(attributes)
                df2.to_parquet(
                    os.path.join(
                        directory_parquet,
                        f"{df2.head(1).cik[0]}-{df2.head(1).accession_number[0]}-{df2.head(1).fdate[0].strftime('%Y-%m-%d')}.parquet"))
                

            else: pd.DataFrame()
        if 'cik' and 'accession_number' in df2.columns: return {'data_load_run':data_load_run,'cik':df2.head(1).cik[0], 'accession_number': df2.head(1).accession_number[0], 'qa':'no'}


In [ ]:
# read list of csvs with variable number of columns
# we define what columns we need 
import pandas as pd
from io import StringIO

# txt = """AB,CD,EF,GH
# foo,20160101,a,1
# foo,20160102,a,3
# foo,20160103,a,5"""

txt2 = """AB,CD,EF
foo,20160101,1
foo,20160102,1
foo,20160103,1"""

# usecols = ['AB', 'CD', 'EF', 'GH','IJ']

# df = pd.read_csv(StringIO(txt2), usecols=lambda c: c in set(usecols))
df = pd.read_csv(StringIO(txt2))
df.EF.head(1).squeeze()



#### A very interesting use of lambda, dictionary comprehension creacion from Andrej Karpathy's code

In [ ]:
import string

string.ascii_letters

stoi = {ch:i for i, ch in enumerate(string.ascii_letters+' ')}
itos = {i:ch for i, ch in enumerate(string.ascii_letters+' ')}

In [ ]:
stoi
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
encode('hii itheerre')
decode(encode('hii itheerre'))

#### Let's practice a bit

In [ ]:
# lambda func to list all files in a given dir
from pathlib import Path
sec_app_parq = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_ALL_PARQ_SEC_APP')

# list_files_in_dir = lambda dir_: sorted(list(Path(dir_).glob('*'))) 
# list_cik_in_dir = lambda dir_: sorted(set([int(file.stem.split('-')[0]) for file in Path(dir_).glob('*')]))
list_fdates_in_dir = lambda dir_: sorted(set([file.stem.split('-', 4)[-1] for file in Path(dir_).glob('*')]))
# list_cik_after = lambda dir_, cik_after: [cik for cik in list_cik_in_dir(dir_) if cik > cik_after ]

In [ ]:
%%time
# list_files(sec_app_parq)[:4]
# list_cik_in_dir(sec_app_parq)[:4]
# list_fdates_in_dir(sec_app_parq)[:4]

min(list_fdates_in_dir(sec_app_parq)), max(list_fdates_in_dir(sec_app_parq))

# print(itemgetter(0, -1)(list_cik_after(sec_app_parq, 1958491)))

# list_cik_in_dir = lambda dir_: sorted(set([int(file.stem.split('-')[0]) for file in Path(dir_).glob('*')]))
# list_cik_after = lambda dir_, cik_after: [cik for cik in list_cik_in_dir(dir_) if cik > cik_after ]
# list_cik_after(sec_app_parq, 1958491)

In [ ]:
# getattr(object, 'y', 5)
# getattr(object, 'x') is completely equivalent to object.x , but getattr(object, 'y', 5), sets a defaut value and won't error out if 'y' doesn't exist

In [ ]:
from operator import itemgetter
l = ['2013-05-20', '2013-05-21', '2013-05-22', '2013-05-23', '2017-05-23']

itemgetter(1, 2)(l)


In [ ]:
falseValue = 345
trueValue  = 65

In [ ]:
(falseValue, trueValue)[bool(True)]

In [ ]:
(lambda: print('I was wrong'), lambda: print('I was right'))[bool(True)]()

In [ ]:
x = print('I was right') if True else rint('I was wrong')

In [ ]:
# ternary operator
for date in l:
    x = 'dropbox' if date <= '2014-01-01' else 'sec_app'
    print(x)

In [ ]:
# another way how to launch programs from python
import subprocess
file = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_ALL_PARQ_SEC_APP/1953154-0001214659-22-013465-2022-11-10.parquet')
# subprocess.call(["ls", "-l"])
subprocess.call(['tad', file])

In [ ]:
# lambda func to list all files in a given dir
from pathlib import Path
sec_app_parq = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_ALL_PARQ_SEC_APP')

In [ ]:
list_file_names = lambda _dir: [file.stem for file in Path(_dir).glob('*')]

In [ ]:
list_file_names(sec_app_parq)[:4]

In [ ]:
from operator import itemgetter
last_date = lambda _dir: [file.stem.split('-', 4)[-1] for file in Path(_dir).glob('*')]

In [ ]:
min(last_date(sec_app_parq)), max(last_date(sec_app_parq))

#### Lambda with two parameters

In [ ]:
list_parq_in_dir = lambda folder, ext: [file.stem for file in Path(folder).glob('*') if file.suffix == ext]

In [ ]:
list_parq_in_dir(sec_app_parq, '.csv')[:4]

In [ ]:
'865623-0000922423-01-500513-2001-07-16'.rsplit('-', 3)[0]

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

list_file_names_csv = lambda folder: [file.stem for file in folder.glob('*.csv')]
list_file_names_parq = lambda folder: [file.stem.rsplit('-', 3)[0] for file in folder.glob('*.parquet')]

In [ ]:
clean_csv_0_2013 = Path(r"/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_ALL_CSV_CLEANED")
clean_parq_0_2013 = Path(r'/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_02_CLEANED_PARQ_0_2013')

csv_files = list_file_names_csv(clean_csv_0_2013)
parquet_files = list_file_names_parq(clean_parq_0_2013)


In [ ]:
len(csv_files), len(parquet_files) 
csv_files[:3]

### Find missing parquet files. It's difference between csv and parquet lists

In [ ]:
l = np.setdiff1d(csv_files, parquet_files)
l_reverse = np.setdiff1d(parquet_files, csv_files)

In [ ]:
[print(name) for name in l if name not in parquet_files]

In [ ]:
df = pd.read_csv(Path.joinpath(clean_csv_0_2013, '1215613-0000905718-03-000066.csv'))

In [ ]:
list_files = lambda folder: [file.stem for file in folder.iterdir()]

clean_parq_0_2013 = Path(r'/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_02_CLEANED_PARQ_0_2013')
clean_parq_2014_2023 = Path('/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_FINAL_PARQ_2014_2023_02_17')

final_parq = Path(f'/Users/yo_macbook/Documents/app_data/TR_02_FINAL_PARQ_BOTH_1999_2023_02_17')
print(f'first part: {len(list_files(clean_parq_0_2013))} + second part: {len(list_files(clean_parq_2014_2023))} = {len(list_files(clean_parq_0_2013)) + len(list_files(clean_parq_2014_2023))}')
print(f'final folder: {len(list_files(final_parq))}')

In [ ]:
%%time
common_files = np.intersect1d(list_files(clean_parq_0_2013), list_files(clean_parq_2014_2023) )
print(len(common_files))

In [ ]:
list_files(clean_parq_0_2013)[:3], list_files(clean_parq_2014_2023)[:3]

In [ ]:
%%time

len(list_files(final_parq))

In [ ]:
%%time
common_elements = list(set(list_files(clean_parq_0_2013)) & set(list_files(clean_parq_2014_2023)))

print(len(common_elements) if common_elements else 0)

In [ ]:
642/482

### Copying file from one folder to another

In [ ]:
from pathlib import Path
import shutil

copy_files = lambda folder_from, folder_to: [shutil.copy(file, folder_to) for file in folder_from.iterdir()]

In [ ]:
%%time
copy_files(clean_parq_2014_2023, final_parq)

In [ ]:
%%time

temp_parq = Path(f'/Users/yo_macbook/Documents/app_data/TR_02_FINAL_PARQ_BOTH_1999_2023_02_17')
final_parq = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_PARQ_ALL')

copy_files(temp_parq, final_parq)

In [ ]:
%%time

temp_failure = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_FAILURE_CASES_PARQ_SEC_APP')
# temp_failure_csv = Path(r'/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_FAILURE_CASES_CSV')
final_failure = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_FAILURE_CASES')

copy_files(temp_failure, final_failure)

In [ ]:
from pathlib import Path
import shutil

copy_files = lambda folder_from, folder_to: [shutil.copy(file, folder_to) for file in Path(folder_from).iterdir()]

In [ ]:
from pathlib import Path
from humanize import naturalsize

# size = final_parq.stat().st_size

# print(naturalsize(size))
final_parq

In [ ]:
size = []
folder_size = lambda folder: size.extend(file.stat().st_size for file in folder.iterdir())
folder_size(final_parq)
print(naturalsize(sum(size)))



In [ ]:
from faker import Faker
np.random.seed(123)

fake = Faker()
n = 10

# categories = [fake.word() for _ in range(3)]

# # Create a list of categories with repetition
# categories = np.repeat(categories, [3, 4, 3])

# Shuffle the categories randomly
np.random.shuffle(categories)

data = {
    'price': np.random.rand(n),
    'amount': np.random.randint(0, 10, n),
    'rank': range(n),
    'date': pd.date_range(start='2022-01-01', periods=n),
    'city': [fake.city() for _ in range(n)],
    'name': [fake.name() for _ in range(n)], 
    'category': np.repeat([fake.country() for _ in range(3)], [3, 4, 3])
}

df1 = pd.DataFrame(data)
df1

In [ ]:
rand_list1 = [fake.name() for _ in range(10)]
rand_list2 = [fake.name() for _ in range(100)]

In [ ]:
len(set(rand_list1) & set(rand_list2) )

In [ ]:
import altair as alt
import pandas as pd
import numpy as np

rand = np.random.RandomState(42)
df = pd.DataFrame({"xval": range(100), "yval": rand.randn(100).cumsum()})

slider = alt.binding_range(min=0, max=100, step=1, name="cutoff:")
selector = alt.selection_single(
    name="SelectorName", fields=["cutoff"], bind=slider, init={"cutoff": 50}
)

alt.Chart(df).mark_point().encode(
    x="xval",
    y="yval",
    color=alt.condition(
        alt.datum.xval < selector.cutoff, alt.value("red"), alt.value("blue")
    ),
).add_selection(selector)

In [ ]:
import altair as alt
from vega_datasets import data

source = data.stocks()
source.symbol.value_counts()
source["category"] = "software"
source.loc[source["symbol"].isin(["AAPL", "IBM"]), "category"] = "hardware"

cat_values = list(source["category"].unique())
sym_values = list(source["symbol"].unique())

cat_options = [None] + cat_values
sym_options = [None] + sym_values

cat_labels = ["All"] + cat_values
sym_labels = ["All"] + sym_values

dropdown_category = alt.binding_select(options=cat_options, labels=cat_labels, name=" ")
dropdown_symbol = alt.binding_select(options=sym_options, labels=sym_labels, name=" ")

selection_category = alt.selection_single(
    fields=["category"], bind=dropdown_category, name="cat"
)
selection_symbol = alt.selection_single(
    fields=["symbol"], bind=dropdown_symbol, name="sym"
)

op_condition = {
    "condition": {
        "test": {"and": [{"selection": "cat"}, {"selection": "sym"}]},
        "value": 1,
    },
    "value": 0.1,
}

chart = (
    alt.Chart(source)
    .mark_line()
    .encode(x="date", y="price", color="symbol", opacity=op_condition)
    .add_selection(selection_symbol, selection_category)
)

chart

In [ ]:
%%time
final_parq_both = Path(r"/Users/yo_macbook/Documents/app_data/TR_02_FINAL_PARQ_BOTH_1999_2023_02_17")

cik_list_parq = sorted(set([file.stem.split('-')[0] for file in final_parq_both.iterdir() if file.suffix == '.parquet']))
cik_list_parq = sorted(set([file.stem.split('-')[0] for file in final_parq_both.iterdir() if file.suffix == '.txt']))
len(cik_list)

In [ ]:
n = 3
[print(el) for index, el in enumerate(set(cik_list)) if index<=n-1]

## List **new** filings to parse

In [ ]:
%%time
from pathlib import Path
from operator import itemgetter

extract_txt_stem = lambda txt_folder: [file.stem for file in txt_folder.glob('*/*.txt')]
extract_parq_stem = lambda parq_folder: ['-'.join(itemgetter(1, 2, 3)(file.stem.split('-'))) for file in parq_folder.iterdir() if file.suffix=='.parquet']

txt_filings_dir = Path(r"/Volumes/fanpc/app_data/sec_apps_data/speed_test/filings_13f_full/filings")
parquet_filings_dir = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_FINAL_PARQ_BOTH_1999_2023_02_17')

txt_file_stems = extract_txt_stem(txt_filings_dir)
parquet_file_stems = extract_parq_stem(parquet_filings_dir)

files_diff = list(set(txt_file_stems) - set(parquet_file_stems))
# filtering out files with `fdate` from years 98, 99 or in general earlier than 2023
files_diff = [file for file in files_diff if int(itemgetter(1)(file.split('-'))) >=23 and int(itemgetter(1)(file.split('-'))) not in [98, 99]]

extract_txt_paths = lambda txt_folder: [file for file in txt_folder.glob('*/*.txt')]
txt_paths_all = extract_txt_paths(txt_filings_dir)


## **New version** of the big mama of the parsing function from `txt` filings to `parquet`

In [23]:

from pathlib import Path
from operator import itemgetter
from datetime import datetime

def parse_txt_filings(txt_folder, parq_folder, dir_failures, data_load_dir, cutoff_date = '2023-02-17'):
    extract_txt_stem = lambda txt_folder: [file.stem for file in txt_folder.glob('*/*.txt')]
    extract_parq_stem = lambda parq_folder: ['-'.join(itemgetter(1, 2, 3)(file.stem.split('-'))) for file in parq_folder.iterdir() if file.suffix=='.parquet']

    txt_file_stems = extract_txt_stem(txt_folder)
    parquet_file_stems = extract_parq_stem(parq_folder)

    files_diff = list(set(txt_file_stems) - set(parquet_file_stems))
    # filtering out files with `fdate` from years 98, 99 or in general earlier than 2023
    files_diff = [file for file in files_diff if int(itemgetter(1)(file.split('-'))) >=23 and int(itemgetter(1)(file.split('-'))) not in [98, 99]]

    extract_txt_paths = lambda txt_folder: [file for file in txt_folder.glob('*/*.txt') if file.stem in files_diff]
    txt_paths_diff = extract_txt_paths(txt_folder)
    print(f'files to parse: {len(txt_paths_diff)}')
    loaded_ciks = []
    data_load_run = datetime.utcnow().strftime("%Y-%m-%d-%H-%M")
    for file in txt_paths_diff:
        data_load_data = end_to_end_parsing(file_path=file, directory_parquet=parq_folder, failures_parq_dir=dir_failures, data_load_run=data_load_run, cutoff_date=cutoff_date)
        loaded_ciks.append(data_load_data)
    loaded_ciks = list(filter(None, loaded_ciks))
    if len(loaded_ciks) > 0: 
        pd.DataFrame(loaded_ciks).to_parquet(data_load_dir.joinpath(data_load_run+'.parquet'))
        
    parse_txt_filings.n_filings_saved = len(loaded_ciks)
    print(f'Total filings written to parquet: {parse_txt_filings.n_filings_saved}')
    
    parse_txt_filings.ciks_saved = list(set(list(map(itemgetter('cik'), loaded_ciks))))
    print(f'cik written to parquet: {parse_txt_filings.ciks_saved}')
    


In [21]:
%%time

txt_filings_dir = Path(r"/Users/yo_macbook/Documents/app_data/TR_01_FILINGS_TXT/filings")
parquet_filings_dir = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_1_FILINGS_PARQ')
failure_cases = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_2_PANDERA_FAILED_ENTRIES')
data_load_dir = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_3_DATA_LOAD_LOGS')

test_parquet = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/test_parq')

parse_txt_filings(txt_filings_dir, parquet_filings_dir, failure_cases, data_load_dir, cutoff_date = '2023-02-17')

files to parse: 96
Total filings written to parquet: 0
Total filings written to parquet: []
CPU times: user 3.03 s, sys: 1.34 s, total: 4.36 s
Wall time: 7.16 s


In [14]:
# parse_txt_filings.n_filings_saved
l = [{'data_load_run': '2023-03-22-15-03',
  'cik': 947484,
  'accession_number': '0000947484-23-000016',
  'qa': 'no'},
    {'data_load_run': '2023-03-22-15-03',
  'cik': 000,
  'accession_number': '0000947484-23-000016',
  'qa': 'no'}]
# print([getattr(d,'cik') for d in l])

# d = {'data_load_run': '2023-03-22-15-03',
#   'cik': 947484,
#   'accession_number': '0000947484-23-000016',
#   'qa': 'no'}
list(map(itemgetter('cik'), l))

[947484, 0]

In [2]:
%%time 
from datetime import datetime
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pandera as pa

############################################################################
############################################################################

## Function to end-to-end parse the raw `txt` filings, write to a pandas dataframe
## and save to a `parquet` file. Only `xml` based filings are dealt with. 
## The filings prior `xml` format are ignored and instead we use the `csv` filings from dropbox

## no division by 1000
## cusip transformed to upper case
## uses `pandera` to remove `cusip` != length of 9 (as non compliant)
## uses `pandera` to remove `values` or `shares` != 0 (as non compliant)
## the code writes bad data into a separate dataframes per filings
## transformation where we group multiple `cusip` into one only after we have removed 
# the ones with zeros in `value` or `shares` 


def end_to_end_parsing(file_path, directory_parquet, failures_parq_dir, data_load_run, cutoff_date = '2023-02-17'):
    """
    func that takes a txt filing's path and parses it to a parquet file
    in the `directory_parquet`. It uses `pandera` lazy mode to filter out
    bad cusip or empty/nan/zero shares or values. Bad records are written
    to the `failures_parq_dir` for a later analysis or correction.
    
    """
    def work_content_extract(file_path):
        """extracting 1. xml or not flag, 2. xml part of filings, SEC header part"""
        open_file = open(file_path, "r")
        file_content = open_file.read()
        open_file.close()

        # find block of text between two words/tags/strings
        start_header = "<SEC-HEADER>"
        end_header = "</SEC-HEADER>"
        block_header_search = re.compile(f"{start_header}.*?{end_header}", re.DOTALL)
        pages_header = re.findall(block_header_search, file_content)

        # check if file is XML or not
        xml_tag_search = re.compile(r"<xml>", flags=re.IGNORECASE)
        if xml_tag_search.search(file_content):
            xml_flag = "YES"

            # xml block search
            start_xml = "<XML\>"
            end_xml = "<\/XML\>"
            block_xml_search = re.compile(f"{start_xml}(.*?){end_xml}", re.DOTALL)
            pages_xml = re.findall(block_xml_search, file_content)

            if len(pages_xml) == 0:
                reportType = ""
                document0 = ""
                document1 = ""
            elif len(pages_xml) == 1:
                document0 = bs(pages_xml[0], "xml")
                coverPage = document0.find("coverPage")
                reportType = coverPage.find("reportType").text.strip()
                document1 = ""
            if len(pages_xml) > 1:
                document0 = bs(pages_xml[0], "xml")
                coverPage = document0.find("coverPage")
                reportType = coverPage.find("reportType").text.strip()
                document1 = bs(pages_xml[1], "xml")

        else:
            xml_flag = "NO"
            pages_xml = []
            reportType = ""
            document1 = ""

        return pages_header, pages_xml, xml_flag, reportType, document1

    def parse_institutionalInvestorInfo(file_path):
        dataDictionary = dict()
        dataDictionary["edgar_path"] = file_path
        periodOfReport = datetime.strptime(
            re.compile(r"(?<=CONFORMED PERIOD OF REPORT:).*")
            .search(pages_header[0])
            .group(0)
            .strip(),
            "%Y%m%d",
        ).date()
        report_Year, report_Quarter = (
            periodOfReport.year,
            (periodOfReport.month - 1) // 3 + 1,
        )

        dataDictionary["accessionNumber"] = (
            re.compile(r"(?<=ACCESSION NUMBER:).*")
            .search(pages_header[0])
            .group(0)
            .strip()
        )
        dataDictionary["cikManager"] = (
            re.compile(r"(?<=CENTRAL INDEX KEY:).*")
            .search(pages_header[0])
            .group(0)
            .strip()
        )
        dataDictionary["managerName"] = (
            re.compile(r"(?<=COMPANY CONFORMED NAME:).*")
            .search(pages_header[0])
            .group(0)
            .strip()
        )
        dataDictionary["periodOfReport"] = periodOfReport
        dataDictionary["report_Quarter"] = report_Quarter
        dataDictionary["report_Year"] = report_Year
        dataDictionary["submissionType"] = (
            re.compile(r"(?<=CONFORMED SUBMISSION TYPE:).*")
            .search(pages_header[0])
            .group(0)
            .strip()
        )
        dataDictionary["filedAsOfDate"] = datetime.strptime(
            re.compile(r"(?<=FILED AS OF DATE:).*")
            .search(pages_header[0])
            .group(0)
            .strip(),
            "%Y%m%d",
        ).date()
        dataDictionary["xml_flag"] = xml_flag
        dataDictionary["created_at"] = datetime.now()
        dataDictionary["updated_at"] = datetime.now()

        # dataDictionary = dict()
        dataDictionary["edgar_path"] = file_path
        if xml_flag == "NO":

            dataDictionary["isAmendment"] = ""
            dataDictionary["amendmentType"] = ""
            dataDictionary["entryTotal"] = int("0" + "")
            dataDictionary["valueTotal"] = float("0.0" + "")
            dt = pd.DataFrame.from_dict([dataDictionary])

        else:
            if len(pages_xml) == 0:
                dataDictionary["isAmendment"] = ""
                dataDictionary["amendmentType"] = ""
                dataDictionary["entryTotal"] = int("0" + "")
                dataDictionary["valueTotal"] = float("0.0" + "")
                dt = pd.DataFrame.from_dict([dataDictionary])

            else:
                document = bs(pages_xml[0], "xml")
                # get sections
                coverPage = document.find("coverPage")
                summaryPage = document.find("summaryPage")
                # get data
                if coverPage.find("isAmendment") is not None:

                    if coverPage.find("isAmendment").text.strip() == "true":
                        dataDictionary["isAmendment"] = coverPage.find(
                            "isAmendment"
                        ).text.strip()
                        if coverPage.find("amendmentType"):
                            dataDictionary["amendmentType"] = coverPage.find(
                                "amendmentType"
                            ).text.strip()
                        else:
                            dataDictionary["amendmentType"] = ""
                    else:
                        dataDictionary["isAmendment"] = coverPage.find(
                            "isAmendment"
                        ).text.strip()
                        dataDictionary["amendmentType"] = ""
                else:
                    dataDictionary["isAmendment"] = ""
                    dataDictionary["amendmentType"] = ""

                if summaryPage is not None:
                    if summaryPage.find("tableEntryTotal").text.strip():
                        dataDictionary["entryTotal"] = int(
                            float(summaryPage.find("tableEntryTotal").text.strip() + "0.0")
                        )
                    else:
                        dataDictionary["entryTotal"] = int("0" + "")


                    if summaryPage.find("tableValueTotal").text.strip():
                        dataDictionary["valueTotal"] = float(
                        summaryPage.find("tableValueTotal").text.strip()
                    + "0.0")

                    else: dataDictionary["valueTotal"] = float("0.0" + "")
                else:
                    dataDictionary["entryTotal"], dataDictionary["valueTotal"] = int(
                        "0" + ""
                    ), float("0.0" + "")

                # create dataframe
                dt = pd.DataFrame.from_dict([dataDictionary])

        return dt

    def parse_institutionalInvestorPortfolio(file_path):
        check = re.compile("13F HOLDINGS REPORT|13F COMBINATION REPORT")
        if check.search(reportType) is not None and len(pages_xml) > 1:

            portfolio = list()
            # find all securities held

            securities = document1.find_all("infoTable")
            for row in securities:
                portfolioDict = dict()
                portfolioDict["edgar_path"] = file_path
                portfolioDict["cusip"] = row.find("cusip").text.strip()
                portfolioDict["nameOfIssuer"] = row.find("nameOfIssuer").text.strip()
                portfolioDict["titleOfClass"] = row.find("titleOfClass").text.strip()
                portfolioDict["sharesValue"] = float(
                    row.find("value").text.strip()
                )
                portfolioDict["sharesHeldAtEndOfQtr"] = int(
                    float(row.find("sshPrnamt").text.strip())
                )

                try:
                    portfolioDict["sharePriceAtEndOfQtr"] = round(
                        portfolioDict["sharesValue"]
                        / portfolioDict["sharesHeldAtEndOfQtr"],
                        2,
                    )
                except ZeroDivisionError:
                    portfolioDict["sharePriceAtEndOfQtr"] = float(int("0" + ""))

                portfolioDict["shares_bonds"] = row.find("sshPrnamtType").text.strip()

                if row.find("putCall") is not None:
                    portfolioDict["putCall"] = row.find("putCall").text.strip()
                else:
                    portfolioDict["putCall"] = ""

                portfolio.append(pd.DataFrame.from_dict([portfolioDict]))

            # concatanate secutires
            dtPortfolio = pd.concat(portfolio, sort=False, ignore_index=True)
        else:
            dtPortfolio = pd.DataFrame.from_dict(
                [
                    {
                        "cusip": "",
                        "nameOfIssuer": "",
                        "titleOfClass": "",
                        "sharesValue": float("0.0" + ""),
                        "sharesHeldAtEndOfQtr": int("0" + ""),
                        "sharePriceAtEndOfQtr": float("0.0" + ""),
                        "shares_bonds": "",
                        "putCall": "",
                        "edgar_path": file_path,
                    }
                ]
            )

        return dtPortfolio

    pages_header, pages_xml, xml_flag, reportType, document1 = work_content_extract(
        file_path
    )
    df1, df2 = parse_institutionalInvestorInfo(
        file_path
    ), parse_institutionalInvestorPortfolio(file_path)

    data = pd.merge(df1, df2, on="edgar_path")
    data = data.assign(dsource='sec_app')    
    data = data.rename(columns={'accessionNumber': 'accession_number', "cikManager": 'cik', "managerName":'cik_name','xml_flag':'type',
           "periodOfReport": 'rdate', "submissionType": 'submission_type', "filedAsOfDate":'fdate', "entryTotal": "entry_total",
           "valueTotal": "value_total", 'putCall':'put_call', "sharesValue":'value', "sharesHeldAtEndOfQtr":'shares',
           "securityType": "security_type", 'titleOfClass':'title_of_class', 'nameOfIssuer':'name_of_issuer' , "edgar_path":'file'})

    column_names = [
        "accession_number",
        "cik",
        "cik_name",
        "rdate",
        "submission_type",
        "fdate",
        "entry_total",
        "value_total",
        "cusip",
        "name_of_issuer",
        "title_of_class",
        "value",
        "shares",
        "shares_bonds",
        "put_call",
        "type",
        "file",
        'dsource',
        "created_at",
        'updated_at',
        'data_load_run'
    ]
    data = data.astype(
        {
            "accession_number": str,
            "cik": "Int64",
            'cik_name': str,
            "rdate": "datetime64[ns]",
            "submission_type": str,
            "fdate": "datetime64[ns]",
            "entry_total": "Int64",
            "value_total": "float64",
            "cusip": str,
            "name_of_issuer": str,
            "title_of_class": str,
            "value": "float64",
            "shares": "float64",
            "shares_bonds": str,
            "put_call": str,
            "type": str,
            "file": str,
            'dsource': str,
            "created_at": "datetime64[ns]",
            "updated_at": "datetime64[ns]"
        }
    )
    attributes = {
        "cik": "first",
        "cik_name": "first",
        "rdate": "first",       
        "fdate": "first",
        "cusip": "first",
        "name_of_issuer": "first",        
        "value": "sum",
        "shares": "sum",
        "title_of_class": "first",
        "shares_bonds": "first",
        "put_call": "first",
        "accession_number": "first",
        "submission_type": "first",
        "type": "first",
        "file": "first",
        'dsource': 'first',
        "entry_total": "first",
        "value_total": "first",
        "created_at": "first",
        "updated_at": "first",
        'data_load_run': 'first'
    }
    data = data.assign(cusip=data.cusip.str.upper(),
                       data_load_run=data_load_run)
    data = data.reindex(columns=column_names)
   
    ## pandera code
    validation_schema = pa.DataFrameSchema({
    "cusip": pa.Column(str,
                       pa.Check(lambda s: s.str.len() == 9),
                       required=True, nullable=False),
    "value":  pa.Column(float, pa.Check(lambda s: s != 0.0), required=True, nullable=False),
    "shares": pa.Column(float, pa.Check(lambda s: s != 0.0), required=True, nullable=False)
        })
    if data.head(1).type.squeeze() != 'NO' and data.head(1).fdate.squeeze() > pd.to_datetime(cutoff_date,format='%Y-%m-%d'):
        try:
            validation_schema.validate(data, lazy=True)
            if not data.empty:
                # group cusips
                df2 = data.groupby(["cusip"], as_index=False).agg(attributes)
                df2.to_parquet(
                    os.path.join(
                        directory_parquet,
                        f"{df2.head(1).cik[0]}-{df2.head(1).accession_number[0]}-{df2.head(1).fdate[0].strftime('%Y-%m-%d')}.parquet",
                                    )
                                )

                # return df2
            else: pd.DataFrame()

        except pa.errors.SchemaErrors as e:
            failure_cases = e.failure_cases
            failure_cases = (failure_cases.assign(df_file=data.file,
                                                 df_cik=data.cik,
                                                 df_accession_number=data.accession_number,
                                                 df_rdate=data.rdate,
                                                 df_fdate=data.fdate,
                                                 df_value=data.value)).astype({'failure_case':str})
            # print(failure_cases)
            failure_cases.to_parquet(Path.joinpath(failures_parq_dir, \
                                                   f"bad-{data.head(1).cik[0]}-{data.head(1).accession_number[0]}-{data.head(1).fdate[0].strftime('%Y-%m-%d')}.parquet"))
            data = data[~data.index.isin(failure_cases["index"])]
            
            if not data.empty:
                # group cusips
                df2 = data.groupby(["cusip"], as_index=False).agg(attributes)
                df2.to_parquet(
                    os.path.join(
                        directory_parquet,
                        f"{df2.head(1).cik[0]}-{df2.head(1).accession_number[0]}-{df2.head(1).fdate[0].strftime('%Y-%m-%d')}.parquet"))
                

            else: pd.DataFrame()
        if 'cik' and 'accession_number' in df2.columns: return {'data_load_run':data_load_run,'cik':df2.head(1).cik[0], 'accession_number': df2.head(1).accession_number[0], 'qa':'no'}


CPU times: user 745 ms, sys: 2 s, total: 2.74 s
Wall time: 931 ms


In [ ]:
s = ['a', 'b']
l = ['a', 'b', 'c', 'z']
if 'b' or 'a' in l: print('all')

In [ ]:
%%time
# file_path = Path(r'/Volumes/fanpc/app_data/sec_apps_data/speed_test/filings_13f_full/filings/1009027/0001009027-23-000001.txt')
file_path = Path(r'/Volumes/fanpc/app_data/sec_apps_data/speed_test/filings_13f_full/filings/1011659/0001085146-23-001425.txt')


txt_filings_dir = Path(r"/Volumes/fanpc/app_data/sec_apps_data/speed_test/filings_13f_full/filings")
parquet_filings_dir = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_PARQ_ALL')
failure_cases = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_FAILURE_CASES')
data_load_run = datetime.utcnow().strftime("%Y-%m-%d-%H-%M")

test_parquet = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/test_parq')

end_to_end_parsing(file_path, test_parquet, failure_cases, data_load_run, cutoff_date = '2023-02-17')

In [ ]:
out = end_to_end_parsing(file_path, test_parquet, failure_cases, data_load_run, cutoff_date = '2023-02-17')
out

In [ ]:
# find duplicates in a list
freq = {}

for element in parquet_files:
    if element in freq:
        freq[element] += 1
    else:
        freq[element] = 1

duplicates = [key for key,value in freq.items() if value > 1 and int(itemgetter(1)(key.split('-'))) >=23 and int(itemgetter(1)(key.split('-'))) not in [98, 99]]
print(duplicates)

In [ ]:
import timeit

data = [
    {"id": 1, "name": "Alice"},
    {"id": 2, "name": None},
    {},
    {"id": 3, "name": "Bob"},
    None,
]

def list_comprehension():
    return [d for d in data if bool(d)]

def filter_function():
    return list(filter(None, data))

print("List comprehension:", timeit.timeit(list_comprehension, number=1000000))
print("Filter function:", timeit.timeit(filter_function, number=1000000))
